# Evolver Loop 4 Analysis

## Problem: Submission Failed with Overlapping Trees in Group 042

The better baseline (70.647306) from snapshot 21165874980 was rejected by Kaggle due to overlapping trees in group 042 (N=42). Our local validation didn't catch this.

**Key Questions:**
1. Why did our local validation miss the overlap?
2. What is the actual overlap in group 042?
3. Can we fix it?
4. What is our best VALID submission?

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
import os

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
BASE_TREE = Polygon(zip(TX, TY))

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    tree = affinity.rotate(BASE_TREE, deg, origin=(0, 0))
    tree = affinity.translate(tree, x, y)
    return tree

print("Functions defined")

In [ ]:
# Load the failed submission
failed_file = '/home/nonroot/snapshots/santa-2025/21165874980/code/submission_candidates/candidate_001.csv'
df = pd.read_csv(failed_file)
print(f"Loaded {len(df)} rows")

In [ ]:
# Check group 042 specifically with STRICTER overlap detection
prefix = '042_'
n_trees = df[df['id'].str.startswith(prefix)]
print(f"N=42 has {len(n_trees)} trees")

polygons = []
for _, row in n_trees.iterrows():
    x = parse_value(row['x'])
    y = parse_value(row['y'])
    deg = parse_value(row['deg'])
    poly = create_tree_polygon(x, y, deg)
    polygons.append((row['id'], poly))

# Check all pairs with VERY strict threshold
overlaps = []
for i in range(len(polygons)):
    for j in range(i+1, len(polygons)):
        id_i, poly_i = polygons[i]
        id_j, poly_j = polygons[j]
        if poly_i.intersects(poly_j):
            inter = poly_i.intersection(poly_j)
            if inter.area > 0:  # ANY overlap
                overlaps.append((id_i, id_j, inter.area))
                
print(f"Found {len(overlaps)} overlaps in N=42:")
for id_i, id_j, area in overlaps:
    print(f"  {id_i} and {id_j}: area = {area:.15f}")

In [ ]:
# Check with even stricter - use buffer to detect near-overlaps
print("\nChecking with buffer for near-overlaps:")
for i in range(len(polygons)):
    for j in range(i+1, len(polygons)):
        id_i, poly_i = polygons[i]
        id_j, poly_j = polygons[j]
        # Check if they're touching or very close
        dist = poly_i.distance(poly_j)
        if dist < 1e-6:
            inter = poly_i.intersection(poly_j)
            print(f"  {id_i} and {id_j}: distance = {dist:.15f}, intersection area = {inter.area:.15f}")

In [ ]:
# Let's check ALL groups for overlaps with zero tolerance
def check_all_overlaps_strict(df):
    overlaps_found = []
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        n_trees = df[df['id'].str.startswith(prefix)]
        if len(n_trees) != n:
            continue
        polygons = []
        for _, row in n_trees.iterrows():
            x = parse_value(row['x'])
            y = parse_value(row['y'])
            deg = parse_value(row['deg'])
            poly = create_tree_polygon(x, y, deg)
            polygons.append((row['id'], poly))
        for i in range(len(polygons)):
            for j in range(i+1, len(polygons)):
                id_i, poly_i = polygons[i]
                id_j, poly_j = polygons[j]
                if poly_i.intersects(poly_j) and not poly_i.touches(poly_j):
                    inter = poly_i.intersection(poly_j)
                    if inter.area > 0:  # ANY positive area
                        overlaps_found.append((n, id_i, id_j, inter.area))
    return overlaps_found

print("Checking ALL groups for overlaps (strict)...")
all_overlaps = check_all_overlaps_strict(df)
print(f"Found {len(all_overlaps)} total overlaps:")
for n, id_i, id_j, area in all_overlaps:
    print(f"  N={n}: {id_i} and {id_j}, area = {area:.15f}")

In [ ]:
# Now let's check our KNOWN GOOD submissions
print("\n" + "="*60)
print("Checking known good submissions:")
print("="*60)

# The 70.658891 submission that passed LB
good_file = '/home/nonroot/snapshots/santa-2025/21164519357/code/exploration/santa-2025.csv'
df_good = pd.read_csv(good_file)
print(f"\nChecking {good_file}...")
good_overlaps = check_all_overlaps_strict(df_good)
print(f"Found {len(good_overlaps)} overlaps")

# The original baseline
baseline_file = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
df_baseline = pd.read_csv(baseline_file)
print(f"\nChecking {baseline_file}...")
baseline_overlaps = check_all_overlaps_strict(df_baseline)
print(f"Found {len(baseline_overlaps)} overlaps")

In [ ]:
# Calculate scores for the good submissions
def calculate_score(df):
    total_score = 0
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        n_trees = df[df['id'].str.startswith(prefix)]
        if len(n_trees) != n:
            continue
        all_coords = []
        for _, row in n_trees.iterrows():
            x = parse_value(row['x'])
            y = parse_value(row['y'])
            deg = parse_value(row['deg'])
            poly = create_tree_polygon(x, y, deg)
            coords = np.array(poly.exterior.coords)
            all_coords.append(coords)
        all_coords = np.vstack(all_coords)
        x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
        y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
        side = max(x_range, y_range)
        total_score += side**2 / n
    return total_score

print("\nScores:")
print(f"Good submission (70.658891): {calculate_score(df_good):.6f}")
print(f"Baseline (70.676102): {calculate_score(df_baseline):.6f}")
print(f"Failed submission: {calculate_score(df):.6f}")

In [ ]:
# Let's look at what the Kaggle validator might be doing differently
# Perhaps they use a different precision or algorithm

# Check the exact coordinates in group 042
print("\nExact coordinates for N=42:")
for _, row in n_trees.head(10).iterrows():
    x = parse_value(row['x'])
    y = parse_value(row['y'])
    deg = parse_value(row['deg'])
    print(f"{row['id']}: x={x:.15f}, y={y:.15f}, deg={deg:.15f}")

In [ ]:
# Let's try using a different overlap detection method
# Kaggle might use a simpler/different algorithm

from shapely.validation import make_valid

def check_overlaps_alternative(df, n):
    """Check overlaps using different methods"""
    prefix = f"{n:03d}_"
    n_trees = df[df['id'].str.startswith(prefix)]
    
    polygons = []
    for _, row in n_trees.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        poly = create_tree_polygon(x, y, deg)
        polygons.append((row['id'], poly))
    
    print(f"\nN={n}: Checking {len(polygons)} trees")
    
    # Method 1: Standard intersection
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            id_i, poly_i = polygons[i]
            id_j, poly_j = polygons[j]
            
            # Check intersection
            if poly_i.intersects(poly_j):
                inter = poly_i.intersection(poly_j)
                if inter.area > 1e-15:  # Very strict
                    print(f"  OVERLAP: {id_i} & {id_j}, area={inter.area:.20f}")
                elif not poly_i.touches(poly_j):
                    print(f"  NEAR: {id_i} & {id_j}, touches={poly_i.touches(poly_j)}, area={inter.area:.20f}")

check_overlaps_alternative(df, 42)

In [ ]:
# Let's check if there's a precision issue with the coordinates
# Maybe the 's' prefix parsing is causing issues

print("\nRaw data for N=42:")
for _, row in n_trees.head(5).iterrows():
    print(f"{row['id']}: x='{row['x']}', y='{row['y']}', deg='{row['deg']}'")

# Check if there are any unusual values
print("\nChecking for unusual values in N=42:")
for _, row in n_trees.iterrows():
    x = parse_value(row['x'])
    y = parse_value(row['y'])
    deg = parse_value(row['deg'])
    if abs(x) > 100 or abs(y) > 100:
        print(f"  Large coordinate: {row['id']}: x={x}, y={y}")
    if deg < 0 or deg > 360:
        print(f"  Unusual angle: {row['id']}: deg={deg}")

In [ ]:
# Let's use a completely different approach - check using the exact same method
# that the competition likely uses

# The competition scoring code probably uses a simple polygon intersection check
# Let's implement it from scratch

def point_in_polygon(point, polygon_coords):
    """Ray casting algorithm"""
    x, y = point
    n = len(polygon_coords)
    inside = False
    j = n - 1
    for i in range(n):
        xi, yi = polygon_coords[i]
        xj, yj = polygon_coords[j]
        if ((yi > y) != (yj > y)) and (x < (xj - xi) * (y - yi) / (yj - yi) + xi):
            inside = not inside
        j = i
    return inside

def get_tree_coords(x, y, deg):
    """Get rotated and translated tree coordinates"""
    import math
    rad = math.radians(deg)
    cos_a = math.cos(rad)
    sin_a = math.sin(rad)
    coords = []
    for tx, ty in zip(TX, TY):
        # Rotate
        rx = tx * cos_a - ty * sin_a
        ry = tx * sin_a + ty * cos_a
        # Translate
        coords.append((rx + x, ry + y))
    return coords

# Check if any vertex of one tree is inside another
def check_vertex_overlap(df, n):
    prefix = f"{n:03d}_"
    n_trees = df[df['id'].str.startswith(prefix)]
    
    trees = []
    for _, row in n_trees.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        coords = get_tree_coords(x, y, deg)
        trees.append((row['id'], coords))
    
    print(f"\nN={n}: Vertex-in-polygon check")
    for i in range(len(trees)):
        for j in range(i+1, len(trees)):
            id_i, coords_i = trees[i]
            id_j, coords_j = trees[j]
            
            # Check if any vertex of i is inside j
            for pt in coords_i:
                if point_in_polygon(pt, coords_j):
                    print(f"  OVERLAP: vertex of {id_i} inside {id_j}")
                    break
            
            # Check if any vertex of j is inside i
            for pt in coords_j:
                if point_in_polygon(pt, coords_i):
                    print(f"  OVERLAP: vertex of {id_j} inside {id_i}")
                    break

check_vertex_overlap(df, 42)